# Code for scrapping data

In [1]:
import AO3
import pandas as pd
import numpy as np
import time

In [2]:
#Testing code snippet to grab titles and summaries to save in dataframe
url = "https://archiveofourown.org/works/11233080"
workid = AO3.utils.workid_from_url(url)
work = AO3.Work(workid)
title = work.title
summary = work.summary

In [22]:
titles = []
summaries = []

for n in range(100, 201):
    time.sleep(15)
    search = AO3.Search(single_chapter = True, word_count=AO3.utils.Constraint(2500, 30000), language = 'en', page = n)
    search.update()
    for result in search.results:
        titles.append(result.title)
        try:
            summaries.append(result.summary)
        except:
            summaries.append('No summary')

In [23]:
df2 = pd.DataFrame(zip(titles, summaries), columns = ['title', 'summary'])

In [25]:
final = pd.concat([df, df2])

In [28]:
final.to_csv('../data/frame_002', index = False)

# Code for cleaning

## Check for duplicate rows

In [65]:
no_dupes = df[~df.duplicated()]

In [55]:
import re
from nltk.tokenize import word_tokenize
import string

punct = list(string.punctuation)

def clean_summaries(text):
    #Get rid of \n at start and end of paragraph
    text = re.sub(r'^\\n|\\n$', '', text)
    #Capture groupings like \xa0
    text = re.sub(r'\\', r' ', text)
    text = re.sub(r'\\(\w+)', '', text)
    #Normalize whitespace
    text = re.sub("\s\s+", " ", text)
    text_token = word_tokenize(text)
    return [w.lower() for w in text_token if w not in punct]

In [30]:
#Check if english title/summary
from langdetect import detect

answer = detect("Sentence to detect")
if answer != 'en':
    #assign 1
else:
    #assign 0
    
#if summary == 'No summary':
    #add flag to ignore row and/or drop

IndentationError: expected an indented block (3121125942.py, line 7)

# Code to train NN to generate new titles/summaries

Merge all summaries together?

In [ ]:
train_len = 4 + 1

text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i - trian_len:i]
    text_sequences.append(seq)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenzier.fit_on_texts(text_sequences)

In [ ]:
sequences = tokenizer.texts_to_sequences(text_sequences)
sequences = np.array(sequences)